In [1]:
from cobra.io import load_model
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion, flux_variability_analysis)

model = load_model("iML1515")

In [2]:
model

Name,iML1515
Memory address,7da569b34620
Number of metabolites,1877
Number of reactions,2712
Number of genes,1516
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [31]:
model.optimize()

,fluxes,reduced_costs
CYTDK2,0.00000,0.000000e+00
XPPT,0.00000,2.775558e-17
HXPRT,0.00000,-1.504671e-02
NDPK5,0.00000,1.445910e-16
SHK3Dr,0.33424,-6.383215e-17
...,...,...
MPTS,0.00000,1.127570e-17
MOCOS,0.00000,-5.642514e-03
BMOGDS2,0.00000,-3.338488e-01
FESD2s,0.00000,1.517883e-18


In [30]:
model.optimize().objective_value

0.8769972144269834

In [38]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.004565,0,0.00%
cl_e,EX_cl_e,0.004565,0,0.00%
cobalt2_e,EX_cobalt2_e,2.192E-05,0,0.00%
cu2_e,EX_cu2_e,0.0006218,0,0.00%
fe2_e,EX_fe2_e,0.01409,0,0.00%
glc__D_e,EX_glc__D_e,10,6,100.00%
k_e,EX_k_e,0.1712,0,0.00%
mg2_e,EX_mg2_e,0.007608,0,0.00%
mn2_e,EX_mn2_e,0.000606,0,0.00%
mobd_e,EX_mobd_e,6.139E-06,0,0.00%


In [36]:
model.medium

{'EX_pi_e': 1000.0,
 'EX_co2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_zn2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_sel_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_tungs_e': 1000.0,
 'EX_slnt_e': 1000.0}

In [ ]:
medium_NoGlc = model.medium
medium_NoGlc["EX_glc__D_e"] = 0.0
model.medium = medium_NoGlc

# There is a small trap here. model.medium can not be assigned to directly. So the following will not work:
# model.medium["EX_co2_e"] = 0.0
# model.medium

model.medium

In [40]:

medium_No02 = model.medium
medium_No02["EX_o2_e"] = 0.0
model.medium = medium_No02

model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.00082,0,0.00%
cl_e,EX_cl_e,0.00082,0,0.00%
co2_e,EX_co2_e,0.05785,1,0.10%
cobalt2_e,EX_cobalt2_e,3.938E-06,0,0.00%
cu2_e,EX_cu2_e,0.0001117,0,0.00%
fe2_e,EX_fe2_e,0.0013,0,0.00%
fe3_e,EX_fe3_e,0.00123,0,0.00%
glc__D_e,EX_glc__D_e,10,6,99.90%
h2o_e,EX_h2o_e,4.591,0,0.00%
k_e,EX_k_e,0.03075,0,0.00%


In [39]:
no_O2 = model.reactions.get_by_id("EX_o2_e")
no_O2.bounds = (0, 1000)
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.00082,0,0.00%
cl_e,EX_cl_e,0.00082,0,0.00%
co2_e,EX_co2_e,0.05785,1,0.10%
cobalt2_e,EX_cobalt2_e,3.938E-06,0,0.00%
cu2_e,EX_cu2_e,0.0001117,0,0.00%
fe2_e,EX_fe2_e,0.0013,0,0.00%
fe3_e,EX_fe3_e,0.00123,0,0.00%
glc__D_e,EX_glc__D_e,10,6,99.90%
h2o_e,EX_h2o_e,4.591,0,0.00%
k_e,EX_k_e,0.03075,0,0.00%


In [43]:
model.metabolites.get_by_id("glc__D_e")

Metabolite identifier,glc__D_e
Name,D-Glucose
Memory address,0x72be16d1b3b0
Formula,C6H12O6
Compartment,e
In 3 reaction(s),"EX_glc__D_e, GLCtex_copy2, GLCtex_copy1"


In [46]:
model.reactions.get_by_id("GLCtex_copy1")

Reaction identifier,GLCtex_copy1
Name,Glucose transport via diffusion (extracellular to periplasm)
Memory address,0x72be07f5b2c0
Stoichiometry,glc__D_e --> glc__D_p D-Glucose --> D-Glucose
GPR,b4036
Lower bound,0.0
Upper bound,1000.0


In [59]:
model.metabolites.get_by_id("atp_c")

Metabolite identifier,atp_c
Name,ATP C10H12N5O13P3
Memory address,0x72be06945d60
Formula,C10H12N5O13P3
Compartment,c
In 349 reaction(s),"ACCOAL, MEPNabcpp, GMHEPAT, RBK, ACKr, PYDAMK, GDPDPK, CYTK2, DADK, ADOCBIK, AACPS1, SUCOAS, AACPS3, CU2abcpp, S2FE2SR, GLUTRS, PG140abcpp, GLYCK, COBALT2abcpp, XYLK2, PYK, DAGK140, GLYK, GLUabcpp,..."


In [75]:
model.reactions.get_by_id("PYK")

Reaction identifier,PYK
Name,Pyruvate kinase
Memory address,0x72be05438770
Stoichiometry,adp_c + h_c + pep_c --> atp_c + pyr_c ADP C10H12N5O10P2 + H+ + Phosphoenolpyruvate --> ATP C10H12N5O13P3 + Pyruvate
GPR,b1854 or b1676
Lower bound,0.0
Upper bound,1000.0


In [ ]:
print(f"Regular model: {model.optimize().objective_value}")

with model:
    double = double_reaction_deletion(model)
    pyk = model.reactions.PYK
    pyk2 = model.reactions.PYK2
    pyk3 = model.reactions.PYK3
    pyk4 = model.reactions.PYK4
    pyk6 = model.reactions.PYK6
    
    print(double.knockout[{pyk, pyk2}, {pyk, pyk3}])


Regular model: 0.87699721425716


In [ ]:
flux_variability_analysis(model, model.reactions.CYTK1)

,minimum,maximum
CYTK1,0.158739,0.158739


In [15]:
flux_variability_analysis(
    model,
    ["CYTK1", "CYTK2", "PYK"],
    loopless=True
)

,minimum,maximum
CYTK1,1.587391e-01,1.587391e-01
CYTK2,-7.855203e-12,3.927601e-12
PYK,0.000000e+00,2.275814e+00


In [ ]:
flux_variability_analysis(
    model,
    ["CYTK1", "CYTK2", "PYK"]
)
# Different results for loopless true/false means that the reaction has a loop that is not possible in vivo


,minimum,maximum
CYTK1,1.587391e-01,1.587391e-01
CYTK2,-4.127310e-12,2.063655e-12
PYK,0.000000e+00,2.275814e+00


In [18]:
model.reactions.get_by_id("CYTK1")

Reaction identifier,CYTK1
Name,Cytidylate kinase (CMP)
Memory address,0x79a4c5974980
Stoichiometry,atp_c + cmp_c <=> adp_c + cdp_c ATP C10H12N5O13P3 + CMP C9H12N3O8P <=> ADP C10H12N5O10P2 + CDP C9H12N3O11P2
GPR,b0910
Lower bound,-1000.0
Upper bound,1000.0


In [10]:
print(f"Regular model: {model.optimize().objective_value}")

with model:
    model.reactions.CYTK1.knock_out() # 0.8752954684189185 <- Solo
    #model.reactions.CYTK2.knock_out() # 0.8769972144269912 <- Solo
    # 0.8752954684189429 both
    print(f"Knock out solution: {model.optimize().objective_value}")

Regular model: 0.876997214427006
Knock out solution: 0.8752954684189185


In [12]:
print(f"Regular model: {model.optimize().objective_value}")

with model:
    model.reactions.GLCtex_copy1.knock_out() #Knocking only one doesn't create a change
    #model.reactions.GLCtex_copy2.knock_out() # Knocking two kills the organism
    print(f"Knocked out solution: {model.optimize().objective_value}")

Regular model: 0.8769972144269661
Knocked out solution: 0.8769972144269661


In [77]:
print(f"Regular model: {model.optimize().objective_value}")

with model:
    model.genes.b1676.knock_out()
    print(f"Knocked out solution: {model.optimize().objective_value}")

Regular model: 0.8769972144269954
Knocked out solution: 0.8769972144269954


In [ ]:
print(f"Regular model: {model.optimize().objective_value}")

with model:
    # double_gene_deletion(model, ["b1676"], ["b1854"])
    # This crashes the program due to no memory. Can't test it
    print(f"Gene knockout: {model.optimize().objective_value}")

Regular model: 0.8769972144269954
Gene knockout: 0.8769972144269954


In [ ]:
print(f"regular model: {model.optimize().objective_value}")

with model:
    single_gene_deletion(model, ["b0910"])
    print(f"Gene knockout solution: {model.optimize().objective_value}")

with model:
    single_reaction_deletion(model, ["CYTK1"])
    print(f"Reaction knockout solution: {model.optimize().objective_value}")

# 0.8752954684189185 <- Was the .knock_out() result

regular model: 0.876997214427006
Gene knockout solution: 0.876997214427006
Reaction knockout solution: 0.876997214427006


In [ ]:
model.summary()
# No carbon source. So no growth
# Gives 'infeasible' error

Infeasible: None (infeasible).

In [19]:
model.exchanges[0:5]

[<Reaction EX_pi_e at 0x72be19608050>,
 <Reaction EX_co2_e at 0x72be19609b80>,
 <Reaction EX_met__L_e at 0x72be1960aba0>,
 <Reaction EX_metsox_S__L_e at 0x72be1960bdd0>,
 <Reaction EX_acgam_e at 0x72be1964d6d0>]

In [20]:
model.demands[0:5]

[<Reaction DM_amob_c at 0x72be1913e270>,
 <Reaction DM_5drib_c at 0x72be185604d0>,
 <Reaction DM_oxam_c at 0x72be1859c590>,
 <Reaction DM_aacald_c at 0x72be182a38c0>,
 <Reaction DM_4crsol_c at 0x72be18060b30>]

In [21]:
model.boundary[0:5]

[<Reaction EX_pi_e at 0x72be19608050>,
 <Reaction EX_co2_e at 0x72be19609b80>,
 <Reaction EX_met__L_e at 0x72be1960aba0>,
 <Reaction EX_metsox_S__L_e at 0x72be1960bdd0>,
 <Reaction EX_acgam_e at 0x72be1964d6d0>]

In [ ]:
flux_variability_analysis(model, model.reactions[:10])
# finds the ranges of each metabolic flux at the optimum

# , fraction_of_optimum=0.9 <- At %90 optimality

,minimum,maximum
CYTDK2,0.000000e+00,2.858328e-12
XPPT,0.000000e+00,-6.226736e-13
HXPRT,0.000000e+00,-1.115665e-12
NDPK5,-2.294751e-02,1.143331e-11
SHK3Dr,3.342403e-01,3.342403e-01
NDPK6,0.000000e+00,-1.269620e-11
NDPK8,-2.294751e-02,1.143331e-11
DHORTS,-2.901063e-01,-2.901063e-01
OMPDC,2.901063e-01,2.901063e-01
PYNP2r,-5.436754e-18,2.294751e-02
